# Collect data from outside sources

https://import.io/ to get school address

In [ ]:
import pandas as pd
import numpy as np
import pygeocoder as pgeo
import os
%matplotlib inline
import matplotlib.pylab as plt
import seaborn as sns

### Prepare data taken from usnews on school which contain addresses - Convert them to Lat Lon

In [ ]:
df_scrape = pd.read_csv('usnews_schools.csv')
df_scrape['address'] = df_scrape.school_value_1.str.lower()+', '+df_scrape.school_value_2.str.lower()
df_scrape['geo'] = df_scrape['address'].apply(lambda x: pgeo.Geocoder.geocode(x).coordinates)
df_scrape['geolat'] = df_scrape['geo'].apply(lambda x: x[0])
df_scrape['geolon'] = df_scrape['geo'].apply(lambda x: x[1]) 
df_scrape['school_link_1/_text'] = df_scrape['school_link_1/_text'].apply(lambda x: x.decode('utf8').encode('ascii','ignore'))
df_scrape_geos = df_scrape[['geolat','geolon','school_link_1/_text']]
df_scrape_geos.columns = ['geolat','geolon','School Name']

In [ ]:
lat1 = minlat
lat2 = maxlat
lon1 = minlon
lon2 = maxlon
df_scrape_geos = df_scrape_geos[(df_scrape_geos.geolat >= lat1) & (df_scrape_geos.geolat <= lat2) &
                                (df_scrape_geos.geolon >= lon1) & (df_scrape_geos.geolon <= lon2)]

### Prepare data files from school csvs 

In [ ]:
def process_scores(fname):
    df_scoress =pd.read_excel(fname)
    df_scoress = df_scoress[(df_scoress.County=='County1')|(df_scoress.County=='County2')]
    return df_scoress


scoressf = [f for f in os.listdir('.') if f[:3]=='xxx']
first = True
df_scores = None
for f in scoressf:
    if first:
        df_scores = process_scores(f)
        first = False
    else:
        tempdf = process_scores(f)
        df_scores = pd.concat([df_scores, tempdf],axis=0)
school_df = pd.merge(df_scores, df_scrape_geos, on='School Name', how = 'inner')    
school_df.replace('*',np.nan,inplace=True)    

### Filter to interesting stats

In [ ]:
school_df = school_df[['geolat','geolon','School Name', 'County', 
                                   'Charter School','Fiscal Year', 'Fiscal Year',
                                   'Math Mean Scale Score','Reading Mean Scale Score',
                                   'Writing Mean Scale Score','Science Mean Scale Score']]
school_df.dropna(axis=0,inplace=True)
school_df = school_df[~school_df['School Name'].apply(lambda x: ('online' in x.lower()) | ('virtual' in x.lower()))]
school_df['Total Score'] = school_df[['Math Mean Scale Score','Reading Mean Scale Score','Writing Mean Scale Score','Science Mean Scale Score']].astype(float).sum(axis=1, numeric_only=float)

### Create dated nn to get associated distances and score

In [ ]:
school_nn10 = KNeighborsRegressor(n_neighbors=1, metric='haversine')
school_nn11 = KNeighborsRegressor(n_neighbors=1, metric='haversine')
school_nn12 = KNeighborsRegressor(n_neighbors=1, metric='haversine')
school_nn13 = KNeighborsRegressor(n_neighbors=1, metric='haversine')
school_nn14 = KNeighborsRegressor(n_neighbors=1, metric='haversine')

In [ ]:
schools = [school_nn10, school_nn11, school_nn12, school_nn13, school_nn14]
dates = [2010, 2011, 2012, 2013, 2014]

#School

In [ ]:
for s, d in zip(schools, dates):
    dftemp = dfschool[(dfschool['Year']==d) & (dfschool['Charter School']=='N')].copy()
    s.fit(dftemp[['geolat','geolon']],dftemp['Total Score'].values)    
with open('school_nn', "wb") as f:
    pickle.dump(schools, f)

### Collect census block information

In [ ]:
def generate_query_grid(lat1 = minlat
                        lat2 = maxlat
                        lon1 = minlon
                        lon2 = maxlon,
                        gridspace = 25.0):
    gridspacelat = abs(lat1-lat2)/gridspace
    gridspacelon = abs(lon2-lon1)/gridspace
    grid = []
    for la in np.arange(lat1, lat2+gridspacelat, gridspacelat):
        for lo in np.arange(lon2-gridspacelon, lon1, gridspacelon):
            bnds = (la, lo)
            grid.append(bnds)
    print 'total generated by grid = {}'.format((gridspace+1)**2)
    return grid

def latlon_to_fips(lat, lon):
    lat = str(np.round(lat,4))
    lon = str(np.round(lon,4))    
    result = requests.get('http://data.fcc.gov/api/block/find?format=json&latitude='+lat+'&longitude='+lon+'&showall=false').json()
    return result['Block']['FIPS']

def extract_fip_from_grid(grid):
    fiplatlon = []
    j=0
    for lat, lon in grid:
        j+=1
        fips = latlon_to_fips(lat, lon)
        fiplatlon.append([fips, lat, lon])
        print j
    return fiplatlon

def break_fips(fips):
    state = fips[0:2]
    county = fips[2:5]
    tract = fips[5:11]
    block_group = fips[11:12]
    block = fips[12:15]
    return state, county, tract, block_group, block

grid = generate_query_grid()
fiplatlon = extract_fip_from_grid(grid)
fiplatlon = np.array(fiplatlon, dtype=object)
dftemp = pd.DataFrame(fiplatlon)
dftemp.columns=['fips','lat','lon']
splitfips = np.array(list(dftemp.fips.apply(break_fips).values),dtype=str)
dfcensus = pd.concat([pd.DataFrame(splitfips, 
             columns=['state', 'county', 'tract', 'block_group', 'block']),
            dftemp],axis=1)
dfcensus = dfcensus[(dfcensus['county']==county1) | (dfcensus['county']==county2)]

In [ ]:
# acs5
median_household_income = 'B19013_001E'
income_per_capita = 'B19301_001E'
median_age = 'B01002_001E'
total_transportation = 'B08301_001E'
car_transportation = 'B08301_002E'
public_transportation = 'B08301_010E'
total_living = 'B25003_001E'
owner_living = 'B25003_002E'
renter_living  = 'B25003_003E'
rent_cost = 'B25064_001E'
# counties
# state
# survey
acs = 'acs5?'
sf = 'sf1?'
# combine
acslist = [median_household_income,income_per_capita,median_age,
           total_transportation,car_transportation,public_transportation,
           total_living,owner_living,renter_living,poptotal]
col =     ['median_household_income','income_per_capita', 'median_age', 
           'total_transportation', 'car_transportation', 'public_transportation',
           'total_living', 'owner_living', 'renter_living', 'poptotal']
acsstr = ','.join(acslist)

In [ ]:
acs_out={}
counties = [county1, county2]
year = ['2010/','2011/','2012/','2013/']
base = 'http://api.census.gov/data/'
key = 'key='
get = '&get='
location = '&for=block+group:*&in=state:04+county:'

for c in counties:
    for y in year:
        q = base+y+acs+key+get+acsstr+location+c
        print q
        r = requests.get(q)
        acs_out[y[:-1]+'_'+c] = jsonloads(r.text)
df_bgrp_census = pd.DataFrame()

for key, item in acs_out.iteritems():
    dftemp = pd.DataFrame(item,dtype=object, columns=col+['state', 'county', 'tract', 'block_group']).ix[1:]
    dftemp['year'] = int(key[0:4])
    df_bgrp_census = pd.concat([df_bgrp_census,dftemp],axis=0)        

In [ ]:
df_census_data = pd.merge(dfcensus, df_bgrp_census, on=['state','county','tract','block_group'], how='left')

In [ ]:
df_census_data['countytract'] = df_census_data.county + df_census_data.tract
df_census_data['countytractblock_group'] = df_census_data.county + df_census_data.tract + df_census_data.block_group

In [ ]:
def nan_fill_with_nn(df, date, metrics = ['geolat','geolon'], years):
    if df.isnull().sum().sum()==0:
        return df
    
    df_good = df.dropna(axis=0, inplace=False).reset_index(drop=True).copy()
    for date in df.year.unique():
        df_bad = df.loc[(df.isnull().sum(axis=1)>0),:].reset_index(drop=True).copy()
        df_bad = [df_bad['year'] == date]
        nn_good = NearestNeighbors(n_neighbors=1).fit(df_good[df_good['year']=='date'][metrics].apply(np.radians))
        key = nn_good.kneighbors(df_bad[metrics].apply(np.radians), return_distance=False)
        key = key.flatten()
        link = pd.DataFrame(key,columns=['key'])
        link = pd.merge(link, df_good, left_on='key',right_index=True, how='left')
        link.drop(['key'],axis=1, inplace=True)
        df_bad.fillna(link,inplace=True)
        df_good = pd.concat([df_good, df_bad], axis=0)    
    return df_good

# CENSUS

In [ ]:
df_census_data  = nan_fill_with_nn(df_census_data )

In [ ]:
df_census_data.reset_index(inplace=True, drop=True)
nn_census = NearestNeighbors(n_neighbors=1).fit(df_census_data['geolat','geolon'])
with open('cenus_nn', "wb") as f:
    pickle.dump(nn_census, f)

In [ ]:
yp = pd.read('yelp_with_census_key.csv')

In [ ]:
nn_census = pd.merge(nn_census, yp, how='right', left_on='key', right_index=True)

In [ ]:
schools = [school_nn10, school_nn11, school_nn12, school_nn13, school_nn14]
dates = [2010, 2011, 2012, 2013, 2014]

In [ ]:
dateschool = dict(zip(dates, schools))

# Add everything together to a posgress database 

In [ ]:
engine = create_engine('''''')
cs=700
data_parcels = pd.read_csv(
    dirc_in+'mls_listings.csv',
    chunksize = cs,
    parse_dates = ['ListDate'],
    low_memory = False)

data_parcels['key'] = nn_census.kneighbors(data_parcels[['geolat','geolon']], return_distance=False)
data_parcels = pd.merge(data_parcels, yp, how='right', left_on='key', right_index=True)
data_parcels = pd.merge(df_census_data, yp, how='right', left_on='key', right_index=True)
data_parcels['distance_from_school'] = kneighbors(data_parcels[['geolat','geolon']])[1]
data_parcels['school_score'] = 0
for d in dates:
    data_parcels.loc[data_parcels['ListDate'].dt.year == d,'school_distance'] = dateschool[d].kneighbors(data_parcels.loc[data_parcels['ListDate'].dt.year == d,'school_score'])[1][0]
    data_parcels.loc[data_parcels['ListDate'].dt.year == d,'school_score'] = dateschool[d].predict(data_parcels.loc[data_parcels['ListDate'].dt.year == d,'school_score'])[1][0]

for i, chunk in enumerate(data_parcels):
    chunk.columns = map(str.lower, chunk.columns)
    chunk.to_sql('listings', engine, if_exists='replace', index=False, chunksize=700)
    print '{} amount of rows have been added to databases'.format(str(cs*(i+1)))
    break 